In [5]:
import pandas as pd
import pymongo
from datetime import datetime

# client = pymongo.MongoClient("mongodb://172.31.99.238:27017")
client = pymongo.MongoClient("mongodb://0.0.0.0:27017") # for without docker
# clinet = pymongo.MongoClient("mongodb://host.docker.internal:27017") # for docker on windows
# client = pymongo.MongoClient("mongodb://172.17.0.1:27017") # for docker on linux (ubuntu)


In [6]:
# Connect to the database
db = client["ad_response_analysis_tf"]

# Retrieve the list of collections (tables)
collections = db.list_collection_names()

# Print the list of collections
print("Collections in the database:")
for collection in collections:
    print(collection)

Collections in the database:
responses_to_ads
survey_respondents
purchase_info
demographic_data
advertisement_info
ad_metrics
ad_demographic_link


In [7]:
# Retrieve the collections (tables) and load them into DataFrames
survey_respondents = pd.DataFrame(list(db["survey_respondents"].find()))
advertisement_info = pd.DataFrame(list(db["advertisement_info"].find()))
responses_to_ads = pd.DataFrame(list(db["responses_to_ads"].find()))

# Print the first few rows of each DataFrame to verify
print("Survey Respondents DataFrame:")
print(survey_respondents.head())

print("\nAdvertisement Info DataFrame:")
print(advertisement_info.head())

print("\nResponses to Ads DataFrame:")
print(responses_to_ads.head())

Survey Respondents DataFrame:
                        _id      Age Education Level  Gender Income Level  \
0  6733d428363969314c8920b5      >60             phd    Male        >100k   
1  6733d428363969314c8920b6      >60         masters  Female        >100k   
2  6733d428363969314c8920b7  30 - 45     high school  Female     50k-100k   
3  6733d428363969314c8920b8  18 - 30             phd    Male        >100k   
4  6733d428363969314c8920b9      >60             phd    Male     50k-100k   

    Location Occupation  RespondentID  
0  Sub-urban     Doctor             1  
1      Urban    Teacher             2  
2      Rural    Teacher             3  
3      Urban    Teacher             4  
4      Urban     Doctor             5  

Advertisement Info DataFrame:
                        _id  AdID  AdCost  AdDuration AdPlatformName  \
0  6733d5cf4eec8b82b12d8693     1     725         445       Facebook   
1  6733d5cf4eec8b82b12d8694     2     466         544        Netflix   
2  6733d5cf4eec8b82b

In [8]:
# Drop _id and respondent_id from survey_respondents
survey_respondents = survey_respondents.drop(columns=['_id'])

# Drop everything except responsetype and adid from responses_to_ads
responses_to_ads = responses_to_ads[['ResponseType', 'AdID', 'RespondentID']]

# Drop everything except adtopic and adid from advertisement_info
advertisement_info = advertisement_info[['AdTopic', 'AdID']]

# Print the first few rows of each DataFrame to verify the changes
print("Survey Respondents DataFrame after dropping columns:")
print(survey_respondents.head())

print("\nResponses to Ads DataFrame after dropping columns:")
print(responses_to_ads.head())

print("\nAdvertisement Info DataFrame after dropping columns:")
print(advertisement_info.head())

Survey Respondents DataFrame after dropping columns:
       Age Education Level  Gender Income Level   Location Occupation  \
0      >60             phd    Male        >100k  Sub-urban     Doctor   
1      >60         masters  Female        >100k      Urban    Teacher   
2  30 - 45     high school  Female     50k-100k      Rural    Teacher   
3  18 - 30             phd    Male        >100k      Urban    Teacher   
4      >60             phd    Male     50k-100k      Urban     Doctor   

   RespondentID  
0             1  
1             2  
2             3  
3             4  
4             5  

Responses to Ads DataFrame after dropping columns:
  ResponseType  AdID  RespondentID
0      Clicked    38           222
1      Ignored    45           203
2      Ignored    19            33
3      Clicked    28           721
4      Clicked    40           513

Advertisement Info DataFrame after dropping columns:
      AdTopic  AdID
0      Sports     1
1  Healthcare     2
2  Technology     3
3   

In [9]:
# Join responses_to_ads with advertisement_info on the basis of adid
responses_ads_info = pd.merge(responses_to_ads, advertisement_info, on='AdID', how='left')

# Print the first few rows of the merged DataFrame to verify the join
print("Merged DataFrame:")
print(responses_ads_info.head())

Merged DataFrame:
  ResponseType  AdID  RespondentID        AdTopic
0      Clicked    38           222        Finance
1      Ignored    45           203    Real Estate
2      Ignored    19            33  Entertainment
3      Clicked    28           721     Healthcare
4      Clicked    40           513         Sports


In [10]:
# Set display options to show the full DataFrame without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Print the full advertisement_info DataFrame
print(advertisement_info)

          AdTopic  AdID
0          Sports     1
1      Healthcare     2
2      Technology     3
3         Fashion     4
4       Political     5
5       Political     6
6       Political     7
7       Political     8
8       Political     9
9       Education    10
10     Technology    11
11      Political    12
12           Food    13
13    Real Estate    14
14      Education    15
15        Fashion    16
16         Sports    17
17     Automobile    18
18  Entertainment    19
19        Finance    20
20     Healthcare    21
21      Education    22
22    Real Estate    23
23      Education    24
24        Finance    25
25     Healthcare    26
26  Entertainment    27
27     Healthcare    28
28     Technology    29
29     Healthcare    30
30         Travel    31
31        Finance    32
32     Healthcare    33
33        Finance    34
34        Finance    35
35        Fashion    36
36        Finance    37
37        Finance    38
38      Political    39
39         Sports    40
40           Foo

In [11]:
# Join survey_respondents with responses_ads_info on RespondentID
final_merged_df = pd.merge(survey_respondents, responses_ads_info, on='RespondentID', how='left')

# Print the first few rows of the final merged DataFrame to verify the join
print("Final Merged DataFrame:")
print(final_merged_df.head())

Final Merged DataFrame:
   Age Education Level Gender Income Level   Location Occupation  \
0  >60             phd   Male        >100k  Sub-urban     Doctor   
1  >60             phd   Male        >100k  Sub-urban     Doctor   
2  >60             phd   Male        >100k  Sub-urban     Doctor   
3  >60             phd   Male        >100k  Sub-urban     Doctor   
4  >60             phd   Male        >100k  Sub-urban     Doctor   

   RespondentID ResponseType  AdID     AdTopic  
0             1      Clicked  69.0        Food  
1             1      Ignored  67.0     Fashion  
2             1      Clicked  58.0  Automobile  
3             1      Clicked  65.0   Education  
4             1      Ignored  41.0        Food  


In [12]:
# Print all the unique RespondentID and their count
unique_respondents = final_merged_df['RespondentID'].value_counts()

print("Unique RespondentID and their count:")
print(unique_respondents)

Unique RespondentID and their count:
RespondentID
620     21
917     21
71      20
110     20
56      19
983     19
310     18
922     18
760     18
802     18
873     18
17      18
536     18
832     18
970     17
461     17
686     17
895     17
584     17
588     17
932     17
645     17
312     17
74      17
138     17
452     17
376     17
820     17
145     17
796     17
453     17
791     17
412     17
672     16
351     16
10      16
684     16
408     16
338     16
208     16
565     16
860     16
641     16
639     16
597     16
78      16
406     16
619     16
222     16
225     16
333     16
773     16
725     16
330     16
766     16
513     16
319     16
113     16
806     16
118     16
969     16
757     16
529     15
594     15
574     15
280     15
90      15
552     15
305     15
919     15
544     15
298     15
295     15
533     15
375     15
756     15
711     15
857     15
41      15
688     15
721     15
777     15
184     15
243     15
801     15
731     14
230 

In [13]:
# Remove all records where ResponseType is 'Ignored'
filtered_df = final_merged_df[final_merged_df['ResponseType'] != 'Ignored']

# Group by RespondentID and AdTopic, then count occurrences
adtopic_counts = filtered_df.groupby(['RespondentID', 'AdTopic']).size().reset_index(name='Count')

# Find the AdTopic with the maximum count for each RespondentID
max_adtopic_counts = adtopic_counts.loc[adtopic_counts.groupby('RespondentID')['Count'].idxmax()]

# Merge back to get the full records
result_df = pd.merge(filtered_df, max_adtopic_counts[['RespondentID', 'AdTopic']], on=['RespondentID', 'AdTopic'], how='inner')

# Print the resulting DataFrame
print("Filtered DataFrame with most frequent AdTopic for each RespondentID:")
print(result_df)

Filtered DataFrame with most frequent AdTopic for each RespondentID:
          Age Education Level  Gender Income Level   Location  \
0         >60             phd    Male        >100k  Sub-urban   
1         >60         masters  Female        >100k      Urban   
2         >60         masters  Female        >100k      Urban   
3     30 - 45     high school  Female     50k-100k      Rural   
4     18 - 30             phd    Male        >100k      Urban   
5     18 - 30             phd    Male        >100k      Urban   
6     18 - 30             phd    Male        >100k      Urban   
7         >60             phd    Male     50k-100k      Urban   
8         >60             phd    Male     50k-100k      Urban   
9     30 - 45         masters    Male      20k-50k  Sub-urban   
10     0 - 18             phd    Male      20k-50k  Sub-urban   
11        >60     high school   Other      20k-50k  Sub-urban   
12    18 - 30         masters  Female        >100k      Urban   
13    18 - 30        

In [14]:
# Group by RespondentID and AdTopic, then count occurrences
grouped_result_df = result_df.groupby(['RespondentID', 'AdTopic']).size().reset_index(name='Count')

# Print the grouped DataFrame
print("Grouped DataFrame with count of each AdTopic for each RespondentID:")
print(grouped_result_df)

Grouped DataFrame with count of each AdTopic for each RespondentID:
     RespondentID        AdTopic  Count
0               1     Automobile      1
1               2      Political      2
2               3        Finance      1
3               4      Political      3
4               5     Healthcare      2
5               6        Fashion      1
6               7        Fashion      1
7               8  Entertainment      1
8               9        Finance      2
9              10           Food      3
10             11        Fashion      1
11             12  Entertainment      1
12             13      Education      1
13             14        Finance      1
14             15        Fashion      1
15             16        Finance      2
16             17        Finance      4
17             18        Fashion      1
18             19  Entertainment      1
19             20     Automobile      1
20             21         Sports      2
21             22  Entertainment      2
22          

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Remove RespondentID and AdID from the DataFrame
features_df = result_df.drop(columns=['RespondentID', 'AdID', 'AdTopic'])

# Encode categorical features
label_encoders = {}
for column in features_df.columns:
    le = LabelEncoder()
    features_df[column] = le.fit_transform(features_df[column])
    label_encoders[column] = le

# Encode the target variable
target_le = LabelEncoder()
target = target_le.fit_transform(result_df['AdTopic'])

# Print the DataFrame after label encoding
print("DataFrame after label encoding:")
print(features_df.head())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_df, target, test_size=0.2, random_state=42)

# Train the Decision Tree model
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Predict the AdTopic for the test set
y_pred = decision_tree.predict(X_test)

# Calculate and print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Decision Tree Model Accuracy: {accuracy:.2f}')

DataFrame after label encoding:
   Age  Education Level  Gender  Income Level  Location  Occupation  \
0    4                3       1             3         1           1   
1    4                2       0             3         2           5   
2    4                2       0             3         2           5   
3    2                1       0             1         0           5   
4    1                3       1             3         2           5   

   ResponseType  
0             0  
1             0  
2             0  
3             0  
4             0  
Decision Tree Model Accuracy: 0.58


In [16]:
from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest model
random_forest = RandomForestClassifier(n_estimators=20, random_state=42)
random_forest.fit(X_train, y_train)

# Predict the AdTopic for the test set
y_pred = random_forest.predict(X_test)

# Calculate and print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Model Accuracy: {accuracy:.2f}')

Random Forest Model Accuracy: 0.61


In [17]:
import joblib
joblib.dump(decision_tree, '/home/vivek/DE-Project/Advertisement-Response-Analysis/Backend/api/models/decision_tree.joblib')

['/home/vivek/DE-Project/Advertisement-Response-Analysis/Backend/api/models/decision_tree.joblib']